In [227]:
import pandas as pd 
import numpy as np 
from sklearn.utils import shuffle

df = pd.DataFrame()
colnames=['Review','Sentiment'] 
df = pd.read_csv('reviews.csv',encoding='utf-8',names=colnames)
df.set_index(['Review', 'Sentiment'])
df = shuffle(df).reset_index(drop=True)

X_train=df.loc[:250,'Review'].values
X_test=df.loc[251:,'Review'].values
Y_train=df.loc[:250,'Sentiment'].values
Y_test=df.loc[251:,'Sentiment'].values
df.head(10)
# print(X_train)

,Review,Sentiment
0,చిత్రం ఖైదీ నెంబర్ 150 నటీనటులు చిరంజీవి - కాజ...,1.0
1,రాంగోపాల్ వర్మ దర్శకత్వంలో యాక్షన్ థ్రిల్లర్ జ...,-1.0
2,ఇండియాలో ఉన్న పేరుమోసిన రాజకీయ నాయకుల బ్లాక్ మ...,1.0
3,మంచు భక్తవత్సలం నాయుడు (మోహన్ బాబు)కు ఇద్దరు భ...,-1.0
4,హర్ష మహేష్ బాబు పాతిక వేల కోట్ల ఆస్తికి వారసుడ...,1.0
5,రూడ్ యార్డ్ కిప్లింగ్ స్క్రీన్ ప్లే జాన్ ఫెవ్ర...,1.0
6,సంజీవ్ రెడ్డి మాటలు నివాస్ స్క్రీన్ప్లే దర్శకత...,1.0
7,గౌతమ్ (రాజ్ తరుణ్) పుట్టుకతోనే అంధుడు అతను ఓ ఆ...,-1.0
8,పూరి జగన్నాథ్ సినిమాలకు తెలుగులో బ్రాండ్ ఇమేజ్...,-1.0
9,ఎర్ర శీను (విజయ్ దేవరకొండ) ఒక దొంగ జీవితంలో స్...,-1.0


In [228]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

tokenizer_obj = Tokenizer()
Total_Reviews = df.loc[:,'Review'].values
tokenizer_obj.fit_on_texts(Total_Reviews)

#pad sequence
max_length= max([len(s.split()) for s in total_reviews])

#size of the vocabulary
vocabulary_size = len(tokenizer_obj.word_index) +1 

X_train_tokens = tokenizer_obj.texts_to_sequences(X_train)
X_test_tokens = tokenizer_obj.texts_to_sequences(X_test)

X_train_pad = pad_sequences (X_train_tokens,maxlen=max_length,padding="post")
X_test_pad = pad_sequences (X_test_tokens,maxlen=max_length,padding="post")

Total_Reviews_tokens = tokenizer_obj.texts_to_sequences(X_train)
Total_reviews_pad = pad_sequences(Total_Reviews_tokens,maxlen=max_length,padding="post")

In [229]:
#traiing the word2vec model
from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.test.utils import get_tmpfile
from gensim import scripts
from smart_open import open
import re
f = open("reviews.txt","r")
data = f.read()

#sg defines if the parameter is skpgram or cbow
sentences_strings_ted = []
for line in data.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

sentences_ted = []
for sent_str in sentences_strings_ted:
	tokens = sent_str.split()
	sentences_ted.append(tokens)

Embedding_size =100 #embedding_size is the size of the embedding vector.

path = get_tmpfile("word2vec.model")
w2v_model = FastText(sentences = sentences_ted,size=100, window=5, min_count=1, workers=4)


#building the vocabulary
vocabulary_words = w2v_model.wv.vocab.keys()
vocabulary = { x:i  for x,i in enumerate(vocabulary_words)}

print(vocabulary)
print(len(vocabulary))

embeddings = 1 * np.random.randn(len(vocabulary_words) + 1,Embedding_size)
embeddings[0] = 0 

for index,word in vocabulary.items():
#     embeddings[index] = np.random.uniform(-0.25,0.25,embedding_dim)
    embeddings[index] = w2v_model.wv[word]
        
    
print(embeddings.shape)





{0: 'చిన్నప్పటి', 1: 'నుంచి', 2: 'గురుకులంలో', 3: 'చదువుకున్న', 4: 'అరవింద్', 5: 'నాని', 6: 'నిజాయితీ', 7: 'మారుపేరుగా', 8: 'పెరుగుతాడు', 9: 'అపరిచితుడులో', 10: 'రామం', 11: 'లాగా', 12: 'నిజాయితీగా', 13: 'ఉండే', 14: 'అతను', 15: 'సమాజంలో', 16: 'ఎన్నో', 17: 'ఇబ్బందులు', 18: 'పడాల్సి', 19: 'వస్తుంది', 20: 'చివరికి', 21: 'వల్ల', 22: 'అతడి', 23: 'ప్రాణమే', 24: 'పోయే', 25: 'పరిస్థితి', 26: 'ఇలాంటి', 27: 'స్థితిలో', 28: 'తన', 29: 'మిత్రులతో', 30: 'కలిసి', 31: 'అవినీతికి', 32: 'వ్యతిరేకంగా', 33: 'పోరాటం', 34: 'మొదలుపెడతాడు', 35: 'అవినీతి', 36: 'పరుల', 37: 'జాబితా', 38: 'తీసి', 39: 'వారిపై', 40: 'న్యాయ', 41: 'పోరాటానికి', 42: 'దిగిన', 43: 'అరవింద్కు', 44: 'పోలికలతో', 45: 'మాయా', 46: 'కన్నన్', 47: 'పెద్ద', 48: 'సమస్య', 49: 'ఎదురవుతుంది', 50: 'అతనేం', 51: 'చేశాడు', 52: 'పోరాటంలో', 53: 'గెలిచాడా', 54: 'లేదా', 55: 'అన్నది', 56: 'మిగతా', 57: 'కథ', 58: 'కథనం', 59: 'అవినీతిపై', 60: 'నేపథ్యంలో', 61: 'గతంలో', 62: 'చాలా', 63: 'సినిమాలు', 64: 'వచ్చాయి', 65: 'జెండాపై', 66: 'కపిరాజుకు', 67: 'కొన్ని', 68: 'సి

In [230]:
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,GRU
from keras.layers.embeddings import Embedding
from gensim.models import Word2Vec
from keras.regularizers import l2


model=Sequential()
model.add(Embedding(len(embeddings), Embedding_size ,input_length=max_length, trainable=True))
model.add(GRU(units=32,dropout = 0.2,recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='softmax'))
model.compile(loss='squared_hinge',
              optimizer='adadelta',
              metrics=['accuracy'])

# model.compile(loss = 'binary_crossentropy' , optimizer = 'adam', metrics = ['accuracy'])

In [231]:
print("Training")

model.fit(X_train_pad,Y_train,epochs =10, batch_size =100, validation_data = (X_test_pad,Y_test),verbose=2)

Training
Train on 251 samples, validate on 17 samples
Epoch 1/10
 - 2s - loss: nan - accuracy: 0.5060 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/10
 - 2s - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/10
 - 2s - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/10
 - 2s - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/10
 - 2s - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 6/10
 - 2s - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 7/10
 - 2s - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 8/10
 - 2s - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 9/10
 - 2s - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 10/10
 - 2s - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00

In [232]:
#one hot encoding and running a SVM classifier

# vocabulary = { i:x  for x,i in enumerate(vocabulary_words)}

# vocabulary = { x:i  for x,i in enumerate(vocabulary_words)}

# X1=np.array([[vocabulary[word] for word in sentence[0]]for sentence in Total_Reviews])
# Y1=np.array(df['Sentiment'].values)
# X_train, X_test, y_train, y_test = train_test_split(X1,Y1, test_size=0.2,random_state=4)

# #Import svm model
# from sklearn import svm

# #Create a svm Classifier
# clf = svm.SVC(kernel='linear') # Linear Kernel

# #Train the model using the training sets
# clf.fit(X_train, y_train)

# #Predict the response for test dataset
# y_pred = clf.predict(X_test)

# from sklearn import metrics

# # Model Accuracy: how often is the classifier correct?
# print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
